In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os, random, shutil, time
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

import sys

from datetime import datetime

In [4]:
starting_dir = os.getcwd()
print (starting_dir)

/Users/kimd999/Dropbox/script/python/srpAnalytics/code/latest/1_tall2wide/3_after_tall2wide/replace_plate_id_in_behavioral/new_use_pandas_and_python


In [53]:
# (goal) load morpho data

morpho_input_file_path = '/Users/kimd999/Dropbox/script/python/srpAnalytics/code/latest/1_tall2wide/3_after_tall2wide/replace_plate_id_in_behavioral/new_use_pandas_and_python/keep/input/zf_morphology_data_335_chemicals_2020DEC16_fixed_wide_DNC_0_full.csv'

df_morpho = pd.read_csv(morpho_input_file_path, header = 0)

df_morpho['plate.id'] = df_morpho['plate.id'].astype('str')

display (len(np.unique(df_morpho['plate.id'])))


#display(df_morpho.head())
#display(df_morpho[df_morpho['chemical.id'] == 163])

# bottle_id_from_here = np.unique(df_morpho['bottle.id'])
# for bottle_id in bottle_id_from_here:
# #    print (bottle_id)
#     if (bottle_id[:2] == "TX"):
#         print (bottle_id)

725

In [65]:
# (goal) load LPR 15 timepoints data

LPR_input_file_path = '/Users/kimd999/Dropbox/script/python/srpAnalytics/code/latest/1_tall2wide/3_after_tall2wide/replace_plate_id_in_behavioral/new_use_pandas_and_python/keep/input/344_zf_LPR_data_phase_1_2_2020JUNE25_updated_plate_id_for_TX_tall_fixed_full_15_timepoints_wide_full.csv'
df_LPR = pd.read_csv(LPR_input_file_path, header = 0)

display(f"before dropna: {df_LPR.shape}")

df_LPR['plate.id'] = df_LPR['plate.id'].astype('str')

df_LPR = df_LPR.dropna(subset=['chemical.id'])

#display (np.unique(df_LPR['chemical.id']))
#df_LPR['chemical.id'] = df_LPR['chemical.id'].astype('str')
#df_LPR['chemical.id'] = df_LPR['chemical.id'].astype('float')
df_LPR['chemical.id'] = df_LPR['chemical.id'].astype('int32')


display(df_LPR.head())

#display(df_LPR[df_LPR['chemical.id']==1031])

#display (len(np.unique(df['chemical.id'])))

df_LPR = df_LPR.dropna(subset=['plate.id'])

display (np.unique(df_LPR['plate.id']))
display (len(np.unique(df_LPR['plate.id'])))
display(f"after dropna: {df_LPR.shape}")


'before dropna: (28416, 20)'

,chemical.id,plate.id,well,chemical_plate_well,conc,t3,t4,t5,t6,t7,t8,t9,t10,t11,t12,t13,t14,t15,t16,t17
0,163,1,A1,163_1_A1,64.0000,0.9,0.0,1.3,0.0,0.3,0.1,0.0,52.0,0.0,0.0,0.0,19.9,20.8,12.6,12.3
1,163,1,A10,163_1_A10,0.0640,31.0,23.2,0.0,0.0,3.3,28.2,0.0,0.0,0.0,0.0,0.0,15.7,50.3,0.0,0.0
2,163,1,A11,163_1_A11,0.0064,4.7,30.7,60.0,38.4,34.7,0.0,26.9,8.8,56.4,9.8,37.1,1.4,50.6,0.0,4.2
3,163,1,A12,163_1_A12,0.0000,68.9,40.8,4.4,1.5,19.5,10.3,16.7,81.1,4.6,18.5,0.6,0.0,0.0,38.3,0.3
4,163,1,A2,163_1_A2,6.4000,0.1,42.9,0.9,90.6,23.4,29.0,49.4,6.2,29.7,59.2,17.5,1.2,25.0,16.3,39.7


array(['1', '2', 'TP962-H10-P1'], dtype=object)

3

'after dropna: (28320, 20)'

In [67]:
# read LPR data as file (not df) and re-write with TP---plate.id
start_time = time.time()
LPR_f_in = open (LPR_input_file_path, "r")

LPR_input_file_path_after_replacement = LPR_input_file_path[:-4] + "_updated_plate_id_with_TP.csv"
LPR_f_out = open(LPR_input_file_path_after_replacement, "w")

for line in LPR_f_in:
    splited_line = line.split(",")
    chemical_id = splited_line[0]
    plate_id = splited_line[1]

    if (str(plate_id) == "plate.id"):
        LPR_f_out.write(line)
    elif (str(plate_id) != "1") and (str(plate_id) != "2"):
        #print (f"plate_id:{plate_id}")
        LPR_f_out.write(line)
    else:
        df_per_chemical = df_morpho[df_morpho['chemical.id'] == int(chemical_id)]
        unique_plate_id_per_chemical = np.unique(df_per_chemical['plate.id'])
        for i in range(len(unique_plate_id_per_chemical)):
#             print (f"\ni:{i}")
#             print(unique_plate_id_per_chemical[i])
#             print(unique_plate_id_per_chemical[i][len(unique_plate_id_per_chemical[i])-1:len(unique_plate_id_per_chemical[i])])
            if ((unique_plate_id_per_chemical[i][len(unique_plate_id_per_chemical[i])-1:len(unique_plate_id_per_chemical[i])]) == plate_id):
                splited_line = line.split(',')
                #print (f"splited_line:{splited_line}")
                new_line = ''
                for j in range(len(splited_line)):
                    #print (f"splited_line[j]:{splited_line[j]}")
                    if j != 1:
                        new_line = new_line + str(splited_line[j].rstrip()) + ","
                    else:
                        new_line = new_line + str(unique_plate_id_per_chemical[i]) + ","
                #print (f"new_line:{new_line}")
                new_line = new_line[:len(new_line)-1] + "\n"
                #print (f"new_line:{new_line}")
                LPR_f_out.write(new_line)
                break

LPR_f_in.close()
LPR_f_out.close()

end_time = time.time()
time_took = str(round((end_time-start_time), 1)) + " seconds"
print ("'read LPR data as file (not df) and re-write with TP---plate.id' took "+str(time_took))
# took 15 seconds

'# read LPR data as file (not df) and re-write with TP---plate.id' took 14.9 seconds


In [69]:
LPR_input_file_path_after_replacement = "/Users/kimd999/Dropbox/script/python/srpAnalytics/code/latest/1_tall2wide/3_after_tall2wide/replace_plate_id_in_behavioral/new_use_pandas_and_python/keep/output/344_zf_LPR_data_phase_1_2_2020JUNE25_full_15_timepoints_wide_full_updated_plate_id_with_TP.csv"

df_LPR_after_replacement = pd.read_csv(LPR_input_file_path_after_replacement, header = 0)

display (df_LPR_after_replacement.shape)

df_LPR_after_replacement['plate.id'] = df_LPR_after_replacement['plate.id'].astype('str')

display (len(np.unique(df_LPR_after_replacement['plate.id'])))
display (np.unique(df_LPR_after_replacement['plate.id']))

is_NaN = df_LPR_after_replacement.isnull()
row_has_NaN = is_NaN.any(axis=1)
rows_with_NaN = df_LPR_after_replacement[row_has_NaN]
display(rows_with_NaN)
#display(df_LPR_after_replacement[df_LPR_after_replacement['plate.id']==nan])


FileNotFoundError: [Errno 2] No such file or directory: '/Users/kimd999/Dropbox/script/python/srpAnalytics/code/latest/1_tall2wide/3_after_tall2wide/replace_plate_id_in_behavioral/new_use_pandas_and_python/keep/output/344_zf_LPR_data_phase_1_2_2020JUNE25_updated_plate_id_for_TX_tall_fixed_full_15_timepoints_wide_full_replaced.csv'